The Office for Budget Responsibility's [March 2024 Economic and Fiscal Outlook](https://obr.uk/docs/dlm_uploads/E03057758_OBR_EFO-March-2024_Web-AccessibleFinal.pdf) updates government forecasts for a five-year budget window for key tax and spending programs, incorporating recent changes in both policy decisions and economic factors. Some parts of PolicyEngine's model are calibrated to be consistent with the OBR's forecasts, and today we're publishing a new version of the model that incorporates the latest information from government publications.

This report contains a high-level and a detailed comparison of PolicyEngine's underlying model against government (principally OBR) and other third-party models.

## UK household outlook

In [1]:
import pandas as pd
from tqdm import tqdm

PE_VARIABLES = [
    "income_tax",
    "total_national_insurance",
    "child_benefit",
    "universal_credit",
    "tax_credits",
    "housing_benefit",
    "ESA_income",
    "income_support",
    "pension_credit",
    "pip",
    "dla",
    "attendance_allowance",
    "state_pension",
    "carers_allowance",
    "fuel_duty",
    "vat",
    "council_tax",
    "household_net_income",
]
df = pd.DataFrame(
    {
        "Year": list(range(2021, 2028)),
    }
)
for var in PE_VARIABLES:
    df[var] = [0] * len(df["Year"])

for year in range(2021, 2028):
    from policyengine_uk import Microsimulation

    sim = Microsimulation()
    for variable in PE_VARIABLES:
        df.loc[df["Year"] == year, variable] = round(
            sim.calculate(variable, year).sum() / 1e9, 1
        )

df["Fuel duties"] = df["fuel_duty"]
df["National Insurance"] = df["total_national_insurance"]
df["Main taxes"] = df["income_tax"] + df["total_national_insurance"]
df["Legacy benefits"] = (
    df["tax_credits"]
    + df["housing_benefit"]
    + df["ESA_income"]
    + df["income_support"]
)
df["Main benefits"] = (
    df["child_benefit"]
    + df["universal_credit"]
    + df["Legacy benefits"]
    + df["pension_credit"]
    + df["pip"]
    + df["dla"]
    + df["attendance_allowance"]
    + df["state_pension"]
    + df["carers_allowance"]
)

pe_df = pd.melt(df, id_vars=["Year"], value_vars=df.columns[1:])
pe_df = pe_df.replace(
    {
        variable.name: variable.label
        for variable in sim.tax_benefit_system.variables.values()
    }
)
pe_df = pe_df.rename(columns={"variable": "Variable", "value": "Value"})


# Data from latest UKMOD country report (in million GBP)
ukmod_df = pd.DataFrame(
    {
        "Year": [2021, 2022, 2023, 2024, 2025, 2026, 2027],
        "Income Tax": [207751, 245771, 303005, 364319, 372215, 383896, 398249],
        "NIC Employees": [63016, 65432, 60613, 55161, 56067, 57567, 59411],
        "NIC Self-employed": [5069, 5377, 5126, 4401, 4479, 4610, 4769],
        "NIC Employers": [
            95576,
            107253,
            110702,
            114677,
            116797,
            120339,
            124771,
        ],
        "Universal Credit": [38091, 40899, 47833, 59290, 69939, 70292, 70531],
        "Child Benefit": [11040, 11150, 11845, 12382, 12677, 12697, 12660],
        "Child and Working Tax Credits": [5797, 5139, 4536, 2565, 0, 0, 0],
        "Housing Benefit": [10212, 9498, 9031, 7098, 5117, 5145, 5159],
        "Income Support": [2523, 2317, 2049, 1199, 0, 0, 0],
        "ESA (income-based)": [4991, 4593, 4124, 2321, 0, 0, 0],
        "Pension Credit": [4574, 4653, 5341, 6483, 6664, 6849, 7026],
        "Personal Independence Payment": [
            13740,
            14170,
            15601,
            16649,
            17204,
            17478,
            17729,
        ],
        "Disability Living Allowance": [
            3713,
            3829,
            4216,
            4499,
            4649,
            4723,
            4791,
        ],
        "Attendance Allowance": [3285, 3388, 3731, 3981, 4114, 4179, 4239],
        "New/basic State Pension": [
            75080,
            77399,
            85229,
            92475,
            95804,
            98199,
            100654,
        ],
        "Second State Pension": [
            18574,
            19150,
            21082,
            22498,
            23247,
            23620,
            23958,
        ],
        "Carers Allowance": [3000, 3093, 3406, 3634, 3755, 3815, 3870],
        "Council Tax": [44047, 45585, 47986, 50093, 52159, 54317, 56562],
    }
)

ukmod_df["National Insurance"] = ukmod_df[
    ["NIC Employees", "NIC Self-employed", "NIC Employers"]
].sum(axis=1)
ukmod_df["Main taxes"] = ukmod_df[
    ["Income Tax", "NIC Employees", "NIC Self-employed", "NIC Employers"]
].sum(axis=1)
ukmod_df["Legacy benefits"] = ukmod_df[
    [
        "Child and Working Tax Credits",
        "Housing Benefit",
        "Income Support",
        "ESA (income-based)",
    ]
].sum(axis=1)
ukmod_df["Main benefits"] = ukmod_df[
    [
        "Universal Credit",
        "Child Benefit",
        "Legacy benefits",
        "Pension Credit",
        "Personal Independence Payment",
        "Disability Living Allowance",
        "Attendance Allowance",
        "Carers Allowance",
        "New/basic State Pension",
        "Second State Pension",
    ]
].sum(axis=1)

ukmod_df = pd.melt(ukmod_df, id_vars=["Year"], value_vars=ukmod_df.columns[1:])
ukmod_df = ukmod_df.rename(columns={"variable": "Variable", "value": "Value"})
ukmod_df.Value = ukmod_df.Value / 1e3

obr_df = pd.DataFrame(
    {
        "Year": [2022, 2023, 2024, 2025, 2026, 2027],
        "Income Tax": [250.5, 279.2, 302.7, 315.8, 331.4, 348.7],
        "National Insurance": [179.2, 168.1, 173.6, 179.8, 186.1, 192.1],
        "Child Benefit": [12.5, 13.5, 13.5, 13.5, 13.5, 13.5],
        "Personal Tax Credits": [7.3, 1.6, -0.1, -0.1, -0.1, -0.1],
        "Housing Benefit (within cap)": [14.8, 13.8, 11.9, 11.9, 11.6, 10.7],
        "Housing Benefit (outside cap)": [0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
        "Universal Credit (within cap)": [40.7, 49.5, 55.0, 57.9, 60.8, 63.7],
        "Universal Credit (outside cap)": [11.1, 17.4, 19.3, 19.7, 20.0, 20.1],
        "DLA and PIP": [28.4, 32.7, 35.3, 38.0, 40.9, 43.4],
        "Attendance Allowance": [6.7, 7.7, 8.0, 8.2, 8.4, 8.6],
        "Pension Credit": [5.4, 5.8, 5.9, 5.7, 5.4, 5.1],
        "State Pension": [124.6, 138.1, 146.0, 150.5, 153.2, 155.1],
        "Fuel duties": [25.1, 24.6, 24.7, 27.3, 27.6, 28.0],
        "VAT": [159.7, 169.6, 173.9, 180.3, 188.0, 195.9],
        "Council Tax": [44.0, 46.3, 48.6, 51.2, 53.9, 56.8],
    }
)

obr_df["Universal Credit"] = obr_df[
    ["Universal Credit (within cap)", "Universal Credit (outside cap)"]
].sum(axis=1)
obr_df["Housing Benefit"] = obr_df[
    ["Housing Benefit (within cap)", "Housing Benefit (outside cap)"]
].sum(axis=1)
obr_df["Main taxes"] = obr_df[["Income Tax", "National Insurance"]].sum(axis=1)
obr_df["Legacy benefits"] = obr_df[
    [
        "Personal Tax Credits",
        "Housing Benefit (within cap)",
        "Housing Benefit (outside cap)",
    ]
].sum(axis=1)
obr_df["Main benefits"] = obr_df[
    [
        "Universal Credit (within cap)",
        "Universal Credit (outside cap)",
        "Child Benefit",
        "DLA and PIP",
        "Attendance Allowance",
        "Pension Credit",
        "State Pension",
    ]
].sum(axis=1)

obr_df = pd.melt(obr_df, id_vars=["Year"], value_vars=obr_df.columns[1:])
obr_df = obr_df.rename(columns={"variable": "Variable", "value": "Value"})

pe_df["Source"] = "PolicyEngine"
ukmod_df["Source"] = "UKMOD"
obr_df["Source"] = "OBR"
df = pd.concat([pe_df, ukmod_df, obr_df])

In [4]:
import plotly.express as px
from policyengine_core.charts import *
from plotly.express.colors import sample_colorscale
import numpy as np

# RF-style 'rainbow' chart.

# Compute average income in each decile for each year


def rainbow_chart(variable: str):
    dist_net_income_df = pd.DataFrame()

    for year in range(2021, 2028):
        sim = Microsimulation()
        net_income = sim.calculate(variable, year) * sim.calculate(
            "inflation_adjustment", year
        )
        decile = sim.calculate("household_income_decile", year)
        avg_by_decile = net_income.groupby(decile).mean()
        avg_by_decile = avg_by_decile.reset_index()
        avg_by_decile["Year"] = year
        dist_net_income_df = pd.concat([dist_net_income_df, avg_by_decile])

    dist_net_income_df = dist_net_income_df[dist_net_income_df.index > 0]
    dist_net_income_df.columns = ["Decile", "Value", "Year"]

    dist_net_income_df_forward_shifted = dist_net_income_df.copy()
    dist_net_income_df_forward_shifted.Year += 1
    dist_net_income_change_df = (
        dist_net_income_df.set_index(["Decile", "Year"])
        / dist_net_income_df_forward_shifted.set_index(["Decile", "Year"])
        - 1
    ).reset_index()
    dist_net_income_change_df = dist_net_income_change_df.dropna()
    dist_net_income_change_df.Decile = dist_net_income_change_df.Decile.astype(
        str
    )
    x = np.linspace(0.1, 0.9, 10)
    c = sample_colorscale("balance", list(x))

    fig = px.bar(
        dist_net_income_change_df,
        x="Year",
        y="Value",
        color="Decile",
        barmode="group",
        color_discrete_map={str(i): c[i - 1] for i in range(1, 11)},
    )

    return format_fig(fig).update_layout(
        title="Figure 1: annual year-on-year change in real household net income forecast by decile",
        height=400,
        yaxis_tickformat="+.0%",
        yaxis_range=[-0.1, 0.1],
        yaxis_title="Relative year-on-year change",
    )

In [5]:
rainbow_chart("household_net_income")

Figure 1 shows PolicyEngine's principal forecast for (real) household disposable incomes, up to and including calendar year 2027. In 2025, we project average relative increases in real disposable income for most income deciles, of between 0 and 3%. In 2026 and 2027, net income falls for most deciles, except for the top four.

All income measures in this report are before housing costs.

In [10]:
def poverty_rate_chart_df():
    pov_df = pd.DataFrame()
    for year in range(2022, 2028):
        sim = Microsimulation()
        in_poverty = sim.calculate("in_poverty", map_to="person", period=year)
        child_poverty = in_poverty[sim.calculate("is_child")].mean()
        wa_poverty = in_poverty[sim.calculate("is_WA_adult")].mean()
        senior_poverty = in_poverty[sim.calculate("is_SP_age")].mean()
        all_poverty = in_poverty.mean()
        pov_df = pd.concat(
            [
                pov_df,
                pd.DataFrame(
                    {
                        "Year": [year],
                        "Children": [child_poverty],
                        "Working-age adults": [wa_poverty],
                        "Seniors": [senior_poverty],
                        "All": [all_poverty],
                    }
                ),
            ]
        )

    return pov_df


pov_df = poverty_rate_chart_df()

In [11]:
fig = px.line(
    pov_df,
    x="Year",
    y=["All", "Children", "Working-age adults", "Seniors"],
    color_discrete_sequence=px.colors.qualitative.T10,
)

fig = format_fig(fig).update_layout(
    yaxis_range=[0, 0.4],
    yaxis_tickformat=".0%",
    title="Figure 2: absolute poverty forecast by age group",
    legend_title="Age group",
    yaxis_title="Percentage",
)
# Add squares to each point

fig.update_traces(mode="markers+lines")

Figure 2 shows the forecast for poverty rates (absolute, before housing costs): increases for all major age groups. With the exception of a reduction in pensioner poverty in 2023 (largely due to benefit increases), we forecast increases in absolute poverty rates for all age groups until at least 2027.

The [latest](https://www.gov.uk/government/statistics/households-below-average-income-for-financial-years-ending-1995-to-2023/households-below-average-income-an-analysis-of-the-uk-income-distribution-fye-1995-to-fye-2023) official absolute poverty rates from the government's Households Below Average Income series (which is for FY2022-23) differs slightly from our 2022 estimates (see Appendix A for a comparison and explanation of senior poverty rate trend differences):

| Age group         | HBAI (%) | PolicyEngine (%) | Difference |
|--------------------|------|--------------|--|
| Children           | 22   | 27           |+5|
| Working-age adults | 15   | 11           |-4|
| Seniors            | 19   | 12           |-5|
| All                | 17   | 15           |-2|



PolicyEngine replicates official forecasts closely, largely due to both calibrating some outputs directly to be consistent (making many of these comparisons - all but VAT and fuel duty - _ex post_), and replicating the use of the OBR's economic determinant forecasts. The next sections evaluate PolicyEngine's model forecasts against OBR forecasts for key programs. We also compare against UKMOD, developed at the University of Essex, using their latest (March 2024) [country report](https://www.iser.essex.ac.uk/research/publications/working-papers/cempa/cempa4-24) as reference.

PolicyEngine’s calibration routine (specified in more detail in our [working paper](https://policyengine.org/uk_data_enhancement.pdf)) calibrates the underlying household survey data weights to reduce inconsistency between the Family Resources Survey and other, more accurate government statistics on taxes and benefits. We use optimisation techniques [like the DWP does](https://assets.publishing.service.gov.uk/media/5a7dddcc40f0b65d88634e32/initial-review-family-resources-survey-weighting-scheme.pdf) in generating the initial FRS household weights, but we include statistics on tax revenues and benefit expenditures as well as demographics instead of excluding them. We also include imputations from other surveys on income and expenditure as part of this process. For a comparison to UKMOD’s approach, see Appendix B.

### Income Tax

PolicyEngine matches Income Tax revenue within 1% in 2024, with this accuracy falling to within 3% in 2027 as we project receipts to grow more slowly than the OBR.

In [6]:
from IPython.display import Markdown


def variable_table(variable):
    table = df[df.Variable.isin([variable]) & (df.Year > 2021)].pivot_table(
        index="Year", columns=["Source"], values="Value"
    )
    table["PolicyEngine / OBR (%)"] = (
        table.PolicyEngine / table.OBR - 1
    ).apply(lambda x: f"{x * 100:+.1f}")
    if "UKMOD" in table.columns:
        table.UKMOD = table.UKMOD.apply(lambda x: round(x, 1))
        table["UKMOD / OBR (%)"] = (table.UKMOD / table.OBR - 1).apply(
            lambda x: f"{x * 100:+.1f}"
        )
    return Markdown(table.to_markdown())


variable_table("Income Tax")

|   Year |   OBR |   PolicyEngine |   UKMOD |   PolicyEngine / OBR (%) |   UKMOD / OBR (%) |
|-------:|------:|---------------:|--------:|-------------------------:|------------------:|
|   2022 | 250.5 |          250.2 |   245.8 |                     -0.1 |              -1.9 |
|   2023 | 279.2 |          279.4 |   303   |                      0.1 |               8.5 |
|   2024 | 302.7 |          299.8 |   364.3 |                     -1   |              20.4 |
|   2025 | 315.8 |          310.1 |   372.2 |                     -1.8 |              17.9 |
|   2026 | 331.4 |          324.2 |   383.9 |                     -2.2 |              15.8 |
|   2027 | 348.7 |          339.3 |   398.2 |                     -2.7 |              14.2 |

### National Insurance

PolicyEngine underestimates total National Insurance receipts by around 6% in 2024, rising to 8% in 2027.

In [7]:
variable_table("National Insurance")

|   Year |   OBR |   PolicyEngine |   UKMOD |   PolicyEngine / OBR (%) |   UKMOD / OBR (%) |
|-------:|------:|---------------:|--------:|-------------------------:|------------------:|
|   2022 | 179.2 |          165.7 |   178.1 |                     -7.5 |              -0.6 |
|   2023 | 168.1 |          164.7 |   176.4 |                     -2   |               4.9 |
|   2024 | 173.6 |          163.8 |   174.2 |                     -5.6 |               0.3 |
|   2025 | 179.8 |          166.8 |   177.3 |                     -7.2 |              -1.4 |
|   2026 | 186.1 |          171.5 |   182.5 |                     -7.8 |              -1.9 |
|   2027 | 192.1 |          176.5 |   189   |                     -8.1 |              -1.6 |

### VAT

PolicyEngine and the OBR match closely despite PolicyEngine not explicitly calibrating to VAT receipts. The difference is less than 1% in 2024 and 2025, and 2% in 2027 (UKMOD does not model VAT so is unavailable for comparison).

In [8]:
variable_table("VAT")

|   Year |   OBR |   PolicyEngine |   PolicyEngine / OBR (%) |
|-------:|------:|---------------:|-------------------------:|
|   2022 | 159.7 |          156   |                     -2.3 |
|   2023 | 169.6 |          165.7 |                     -2.3 |
|   2024 | 173.9 |          175.6 |                      1   |
|   2025 | 180.3 |          181.2 |                      0.5 |
|   2026 | 188   |          186.7 |                     -0.7 |
|   2027 | 195.9 |          193.2 |                     -1.4 |

### Council Tax

We match Council Tax receipts closely (as does UKMOD)- Council Tax is imputed but relying on reported values by region (rather than simulated across local authorities), and all three models use the same OBR uprating forecasts to increase year-by-year.

In [9]:
variable_table("Council Tax")

|   Year |   OBR |   PolicyEngine |   UKMOD |   PolicyEngine / OBR (%) |   UKMOD / OBR (%) |
|-------:|------:|---------------:|--------:|-------------------------:|------------------:|
|   2022 |  44   |           42   |    45.6 |                     -4.5 |               3.6 |
|   2023 |  46.3 |           45.2 |    48   |                     -2.4 |               3.7 |
|   2024 |  48.6 |           47.9 |    50.1 |                     -1.4 |               3.1 |
|   2025 |  51.2 |           50.4 |    52.2 |                     -1.6 |               2   |
|   2026 |  53.9 |           53.3 |    54.3 |                     -1.1 |               0.7 |
|   2027 |  56.8 |           55.9 |    56.6 |                     -1.6 |              -0.4 |

### Fuel duty

Fuel duty is matched by around 10% on average, but is more volatile in PolicyEngine's model than the OBR forecast (UKMOD does not model fuel duty so cannot be used for comparison).

The deviation is likely caused by the fact that the OBR accounts for the impact of car electrification on revenues, unlike PolicyEngine which does not.


In [10]:
variable_table("Fuel duties")

|   Year |   OBR |   PolicyEngine |   PolicyEngine / OBR (%) |
|-------:|------:|---------------:|-------------------------:|
|   2022 |  25.1 |           24.1 |                     -4   |
|   2023 |  24.6 |           23.5 |                     -4.5 |
|   2024 |  24.7 |           28   |                     13.4 |
|   2025 |  27.3 |           29.5 |                      8.1 |
|   2026 |  27.6 |           30.9 |                     12   |
|   2027 |  28   |           33.3 |                     18.9 |

### Universal Credit

PolicyEngine's Universal Credit forecast is around 4% higher in 2024 than the OBR's, rising to 7.5% in 2027. This likely reflects uncertainty over the rollout of Universal Credit from legacy benefits.

In [11]:
variable_table("Universal Credit")

|   Year |   OBR |   PolicyEngine |   UKMOD |   PolicyEngine / OBR (%) |   UKMOD / OBR (%) |
|-------:|------:|---------------:|--------:|-------------------------:|------------------:|
|   2022 |  51.8 |           57.6 |    40.9 |                     11.2 |             -21   |
|   2023 |  66.9 |           68.7 |    47.8 |                      2.7 |             -28.6 |
|   2024 |  74.3 |           77.6 |    59.3 |                      4.4 |             -20.2 |
|   2025 |  77.6 |           83   |    69.9 |                      7   |              -9.9 |
|   2026 |  80.8 |           87.2 |    70.3 |                      7.9 |             -13   |
|   2027 |  83.8 |           90.1 |    70.5 |                      7.5 |             -15.9 |

### Legacy benefits

We match the OBR's forecasts for legacy benefit spending by around 7% on average.

In [12]:
variable_table("Legacy benefits")

|   Year |   OBR |   PolicyEngine |   UKMOD |   PolicyEngine / OBR (%) |   UKMOD / OBR (%) |
|-------:|------:|---------------:|--------:|-------------------------:|------------------:|
|   2022 |  22.2 |           24.2 |    21.5 |                      9   |              -3.2 |
|   2023 |  15.4 |           17.2 |    19.7 |                     11.7 |              27.9 |
|   2024 |  11.8 |           13.4 |    13.2 |                     13.6 |              11.9 |
|   2025 |  11.8 |           11.9 |     5.1 |                      0.8 |             -56.8 |
|   2026 |  11.5 |           12.3 |     5.1 |                      7   |             -55.7 |
|   2027 |  10.6 |           11.7 |     5.2 |                     10.4 |             -50.9 |

### Child Benefit

We match Child Benefit spending by around 10% on average.

In [13]:
variable_table("Child Benefit")

|   Year |   OBR |   PolicyEngine |   UKMOD |   PolicyEngine / OBR (%) |   UKMOD / OBR (%) |
|-------:|------:|---------------:|--------:|-------------------------:|------------------:|
|   2022 |  12.5 |           12.6 |    11.2 |                      0.8 |             -10.4 |
|   2023 |  13.5 |           14.4 |    11.8 |                      6.7 |             -12.6 |
|   2024 |  13.5 |           15   |    12.4 |                     11.1 |              -8.1 |
|   2025 |  13.5 |           15   |    12.7 |                     11.1 |              -5.9 |
|   2026 |  13.5 |           14.9 |    12.7 |                     10.4 |              -5.9 |
|   2027 |  13.5 |           15   |    12.7 |                     11.1 |              -5.9 |

### Pension Credit

Pension Credit sees the most divergence between PolicyEngine and the OBR of all simulated programs: £3bn on average, which represents aroud 40% of the size of the OBR forecast. UKMOD also shows differences up to 40% (but lower overall). Given that Pension Credit affects the pensioners with extremely low incomes (whose data in the FRS is less reliable), this is an area for further investigation.

In [14]:
variable_table("Pension Credit")

|   Year |   OBR |   PolicyEngine |   UKMOD |   PolicyEngine / OBR (%) |   UKMOD / OBR (%) |
|-------:|------:|---------------:|--------:|-------------------------:|------------------:|
|   2022 |   5.4 |            9.3 |     4.7 |                     72.2 |             -13   |
|   2023 |   5.8 |            5.2 |     5.3 |                    -10.3 |              -8.6 |
|   2024 |   5.9 |            2.9 |     6.5 |                    -50.8 |              10.2 |
|   2025 |   5.7 |            3.1 |     6.7 |                    -45.6 |              17.5 |
|   2026 |   5.4 |            3.6 |     6.8 |                    -33.3 |              25.9 |
|   2027 |   5.1 |            4.2 |     7   |                    -17.6 |              37.3 |

### State Pension

In 2024, we match the OBR on State Pension forecast within 7%, though this narrows to within 2% in 2026 and 2027.

In [15]:
variable_table("State Pension")

|   Year |   OBR |   PolicyEngine |   PolicyEngine / OBR (%) |
|-------:|------:|---------------:|-------------------------:|
|   2022 | 124.6 |          125.1 |                      0.4 |
|   2023 | 138.1 |          148.2 |                      7.3 |
|   2024 | 146   |          155.9 |                      6.8 |
|   2025 | 150.5 |          157.5 |                      4.7 |
|   2026 | 153.2 |          155.9 |                      1.8 |
|   2027 | 155.1 |          153.5 |                     -1   |

## Appendix: senior poverty rates

We project a significantly lower senior poverty estimate for 2023 than HBAI largely because of our modelling of the Energy Price Guarantee. We modelled this as an increase to incomes, rather than a decrease in consumption. Figure 3 shows the senior poverty rate with and without this intervention.

In [12]:
from policyengine_core.reforms import Reform

no_epg_subsidy = Reform.from_dict(
    {
        "gov.ofgem.energy_price_guarantee": {
            "year:2020:10": 1_000_000,
        }
    },
    country_id="uk",
)


def poverty_rate_chart_df():
    pov_df = pd.DataFrame()
    for year in range(2022, 2028):
        baseline = Microsimulation()
        sim = Microsimulation(reform=no_epg_subsidy)
        in_poverty_b = baseline.calculate(
            "in_poverty", map_to="person", period=year
        )
        senior_poverty_b = in_poverty_b[baseline.calculate("is_SP_age")].mean()
        in_poverty_no_epg = sim.calculate(
            "in_poverty", map_to="person", period=year
        )
        senior_poverty_no_epg = in_poverty_no_epg[
            sim.calculate("is_SP_age")
        ].mean()
        pov_df = pd.concat(
            [
                pov_df,
                pd.DataFrame(
                    {
                        "Year": [year],
                        "Baseline": [senior_poverty_b],
                        "No EPG subsidy": [senior_poverty_no_epg],
                    }
                ),
            ]
        )

    return pov_df


pov_df = poverty_rate_chart_df()

In [18]:
fig = px.line(
    pov_df,
    x="Year",
    y=["Baseline", "No EPG subsidy"],
    color_discrete_sequence=[px.colors.qualitative.T10[3]] * 2,
)

fig = format_fig(fig).update_layout(
    yaxis_range=[0, 0.4],
    yaxis_tickformat=".0%",
    title="Figure 3: senior absolute poverty forecast under alternative policy scenarios",
    legend_title="Policy",
    yaxis_title="Percentage",
)
# Add squares to each point

fig.update_traces(mode="markers+lines")

# Make the second line dashed

fig.data[1].line.dash = "dash"

fig

## Appendix B: modelling assumptions

Both PolicyEngine and UKMOD face choices in how much to calibrate their models to administrative aggregates. These choices can involve inheriting calibration from the Department for Work and Pensions' (DWP) microdata calibration, applying uprating factors, and determining benefit take-up rates.

PolicyEngine has opted for a broader-scale calibration approach. We use optimisation techniques similar to those employed by the DWP when generating the initial Family Resources Survey (FRS) household weights. However, while the DWP primarily focuses on calibrating to demographic totals, PolicyEngine also includes statistics on tax revenues and benefit expenditures in the calibration process. Furthermore, we incorporate imputations from other surveys on income and expenditure to refine the microdata.

In contrast, UKMOD relies on the original FRS weights calibrated by the DWP to demographic totals, and limits adjustments to applying uprating factors and policy-related mechanical changes, such as transitioning households from legacy benefits to Universal Credit, adjusting unemployment status, and modelling benefit take-up rates.

While both approaches have their merits, we believe that microsimulation models should, at a minimum, strive to replicate the reported administrative totals for years where this information is known. PolicyEngine's more extensive calibration process aims to achieve this goal and improve the model's accuracy in estimating the impacts of policy reforms on UK households.

It is important to acknowledge that the level of confidence in projections may decrease for future years due to inherent uncertainties. However, calibrating the model to match known administrative data ensures a solid foundation for estimating policy reform impacts. Furthermore, we believe OBR projections are the best sources available for calibrating microdata in future years.

We recognise that the choice of calibration approach can influence comparisons between different microsimulation models. By providing transparency about our modelling assumptions and calibration methods, we hope to facilitate a better understanding of the similarities and differences between PolicyEngine and other models, such as UKMOD.

As we continue to develop and refine our model, we remain committed to striking a balance between model accuracy and transparency, while acknowledging the uncertainties associated with future projections.